# Train an IntegratedML model on Readmission Dataset
## Use JDBC to connect to InterSystems IRIS database
This Notebook demonstrates:
- Using the JayDeBeApi Python library to connect to InterSystems IRIS
- Creating views to segment data into training and test sets
- Defining and training an IntegratedML model to predict marketing campaign responses
- Comparing the resulting model's predictions to data in the test set (that the model was not trained on)
- Using the IntegratedML "VALIDATE MODEL" command to calculate accuracy metrics on the test set data

In [ ]:
!conda activate
!pip3 install jupyterthemes

In [ ]:
!jt -r

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

!ls /tf/intersystems-jdbc-3.1.0.jar
RUN apt-get update && \
    DEBIAN_FRONTEND=noninteractive \
    apt-get -y install default-jre-headless && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

In [ ]:
!pip install JayDeBeApi

In [ ]:
!pip list JPype1

### 1. Set environment variables, if necessary

In [ ]:
#import os
#os.environ['JAVA_HOME']='C:\Progra~1\Java\jdk1.8.0_241'
#os.environ['CLASSPATH'] = 'C:\interSystems\IRIS20194\dev\java\lib\JDK18\intersystems-jdbc-3.0.0.jar'
#os.environ['HADOOP_HOME']='C:\hadoop\bin'  #winutil binary must be in Hadoop's Home

### 2. Get jdbc connection and cursor

In [3]:

import jaydebeapi
url = "jdbc:IRIS://irisimlsvr:1972/USER" #"jdbc:IRIS://172.17.0.1:8091/USER"
driver = 'com.intersystems.jdbc.IRISDriver'
user = "SUPERUSER"
password = "SYS"
#libx = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18"
#jarfile = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18/intersystems-jdbc-3.0.0.jar"
jarfile = "./intersystems-jdbc-3.1.0.jar"

In [4]:
conn = jaydebeapi.connect(driver, url, [user, password], jarfile)
curs = conn.cursor()

java.sql.SQLException: java.sql.SQLException: [InterSystems IRIS JDBC] Communication link failure: Connection refused (Connection refused)

### 3. specify the source data table

In [5]:
dataTable = 'SQLUser.Readmission'

### 4. Execute a query and display results in Pandas DataFrame

In [ ]:
import pandas as pd
from IPython.display import display

df = pd.read_sql("select TOP 20 * from %s" % dataTable, conn)
display(df)

In [7]:
# Show number rows
df1 = pd.read_sql("SELECT COUNT(*) FROM %s" % dataTable, conn)
display(df1)

,Aggregate_1
0,14047


### 5. Make some views to split training and testing datasets

In [8]:
df1 = pd.read_sql("SELECT COUNT(*) FROM %s WHERE  ID>=11000" % dataTable, conn)
display(df1)

,Aggregate_1
0,3048


In [ ]:
# Training set view
curs.execute("CREATE VIEW SQLUser.ReadmissionTraining AS SELECT * FROM SQLUser.Readmission WHERE ID<11000")
# Prediction set
curs.execute("CREATE VIEW SQLUser.ReadmissionPredict AS SELECT * FROM SQLUser.Readmission WHERE ID>=11000")

### 6. Look at Data

In [ ]:
display(pd.read_sql("select TOP 20 * from SQLUser.ReadmissionTraining", conn))

In [43]:
display(pd.read_sql("SELECT COUNT(*) rowCount FROM SQLUser.ReadmissionTraining", conn))

,rowCount
0,10999


In [42]:
df.dtypes

ID                                                                                          int64
AdmitReason                                                                                object
Com_ANY_MALIGNANCY_INCLUDING_LYMPHOMA_AND_LEUKEMIA_EXCEPT_MALIGNANT_NEOPLASM_OF_SKIN        int64
Com_CEREBROVASCULAR_DISEASE                                                                 int64
Com_CHRONIC_PULMONARY_DISEASE                                                               int64
Com_DEMENTIA                                                                                int64
Com_DIABETES_WITHOUT_CHRONIC_COMPLICATION                                                   int64
Com_HEMIPLEGIA_OR_PARAPLEGIA                                                                int64
Com_MILD_LIVER_DISEASE                                                                      int64
Com_MYOCARDIAL_INFARCTION                                                                   int64
Com_PERIPHERY_VASCUL

In [41]:
len(df.dtypes)

46

### 7. Choose ML framework with "SET ML CONFIGURATION"


In [44]:
curs.execute("SET ML CONFIGURATION %AutoML")

### 8. Create and Train an IntegratedML Model using default settings
IntegratedML only needs a model name, the name of the column that is the target column to predict, and a table (or SELECT query to specify input columns.

This is a simple DDL query that executes immediately.

In [37]:
curs.execute("DROP MODEL ReadmissionModel1")
res = curs.execute("CREATE MODEL ReadmissionModel1 PREDICTING (MxWillReAdmit) FROM ReadmissionTraining")

Now that the model is defined, you can TRAIN it, which invokes the AutoML machine learning procedure.

In [39]:
curs.execute("TRAIN MODEL ReadmissionModel")

Once that finishes, you can see some information about the model in the "ML_TRAINED_MODELS" table.

In [46]:
df3 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", conn)
display(df3)

,MODEL_NAME,TRAINED_MODEL_NAME,PROVIDER,TRAINED_TIMESTAMP,MODEL_TYPE,MODEL_INFO
0,ReadmissionModel,ReadmissionModel2,AutoML,2020-07-24 14:51:55.344000,classification,"ModelType:Random Forest, Package:sklearn, Prob..."
1,ReadmissionModel,ReadmissionModel4,AutoML,2020-10-19 04:19:46.582000,classification,"ModelType:Random Forest, Package:sklearn, Prob..."


### 9. Compare model output to data it has not seen yet
Now you can use SQL to SELECT data from another table, run the IntegratedML model on this new data, and see how well the predictions match the data!

In [19]:
df4 = pd.read_sql("SELECT PREDICT(ReadmissionModel) AS PredictedReadmit, \
                  MxWillReAdmit AS ActualReadmit FROM ReadmissionPredict", conn)
display(df4)

,PredictedReadmit,ActualReadmit
0,1,1
1,1,1
2,0,0
3,1,0
4,1,0
...,...,...
3043,0,0
3044,0,0
3045,0,0
3046,0,0


### 10. VALIDATE MODEL command calculates accuracy metrics
You can certainly take that output above and calculate the accuracy using a standard formula, but IntegratedML has a built-in function to do that!

Each time you run the command "VALIDATE MODEL..." it generates a set of metrics calculated on the data passed into the query. Since this table can be a bit difficult to read in its raw form we use a simple "pivot" call to arrange the data.

In [47]:
curs.execute("VALIDATE MODEL ReadmissionModel FROM SQLUser.ReadmissionPredict")
df5 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS", conn)
df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df6)

METRIC_NAME,Accuracy,F-Measure,Precision,Recall
VALIDATION_RUN_NAME,,,,
ReadmissionModel21,0.82,0.59,0.63,0.55
ReadmissionModel41,0.82,0.59,0.64,0.56


### 9. Query that highlights incorrectly predicted rows

This query retrieves encounters the model incorrectly scored, along with the probability estimate of the prediction.

In [50]:
display(pd.read_sql('''
SELECT
	ID,
	PROBABILITY(ReadmissionModel FOR '1') ReadmissionProbability,
	PREDICT(ReadmissionModel) PredictedReadmission,
	MxWillReAdmit ActualReadmission,
	*
FROM
	ReadmissionPredict
WHERE
	MxWillReAdmit = 1
	AND MxWillReAdmit != PREDICT(ReadmissionModel)
''', conn))

,ID,ReadmissionProbability,PredictedReadmission,ActualReadmission,ID,AdmitReason,Com_ANY_MALIGNANCY_INCLUDING_LYMPHOMA_AND_LEUKEMIA_EXCEPT_MALIGNANT_NEOPLASM_OF_SKIN,Com_CEREBROVASCULAR_DISEASE,Com_CHRONIC_PULMONARY_DISEASE,Com_DEMENTIA,...,MxNumAdmitsSixMonth,MxNumAdmitsThreeMonth,MxNumAdmitsTwelveMonth,MxNumDiagOneMonth,MxNumDiagTwelveMonth,MxSmoker,MxStartDateDayOfMonth,MxStartDateDayOfWeek,MxStartDateMonth,MxWillReAdmit
0,11008,0.332644,0,1,11008,Encounter for problem,1,0,1,0,...,5,3,11,0,0,0,22,4,10,1
1,11009,0.332644,0,1,11009,Encounter for problem,1,0,1,0,...,5,3,11,0,0,0,19,4,11,1
2,11015,0.190000,0,1,11015,SNOMED Code,0,0,0,0,...,0,0,0,1,2,0,22,3,1,1
3,11028,0.010000,0,1,11028,SNOMED Code,0,0,0,0,...,0,0,0,1,1,0,3,4,11,1
4,11046,0.010000,0,1,11046,SNOMED Code,0,0,1,0,...,0,0,0,1,1,0,3,7,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,14036,0.130000,0,1,14036,Emergency Room admission (procedure),0,0,0,0,...,3,3,3,2,4,0,14,5,11,1
314,14037,0.140000,0,1,14037,Emergency Room admission (procedure),0,0,0,0,...,4,4,4,2,5,0,14,7,12,1
315,14038,0.110000,0,1,14038,Emergency Room admission (procedure),0,0,0,0,...,5,3,5,2,6,0,13,2,1,1
316,14040,0.230000,0,1,14040,Emergency Room admission (procedure),0,0,0,0,...,0,0,0,0,0,0,14,7,12,1


### 9. Query that pulls recent Encounters and filters by those with high probability of readmission

In [48]:
display(pd.read_sql('''
SELECT 
	ID,
	PROBABILITY(ReadmissionModel FOR '1') ReadmissionProbability,
	PREDICT(ReadmissionModel) PredictedReadmission,
	*
FROM
	ReadmissionPredict
WHERE
	MxEncounterEndYear >=2019 AND
	MxEndDateMonth >= 12 AND
    PROBABILITY(ReadmissionModel FOR '1') >= 0.25
    ''', conn))

,ID,ReadmissionProbability,PredictedReadmission,ID,AdmitReason,Com_ANY_MALIGNANCY_INCLUDING_LYMPHOMA_AND_LEUKEMIA_EXCEPT_MALIGNANT_NEOPLASM_OF_SKIN,Com_CEREBROVASCULAR_DISEASE,Com_CHRONIC_PULMONARY_DISEASE,Com_DEMENTIA,Com_DIABETES_WITHOUT_CHRONIC_COMPLICATION,...,MxNumAdmitsSixMonth,MxNumAdmitsThreeMonth,MxNumAdmitsTwelveMonth,MxNumDiagOneMonth,MxNumDiagTwelveMonth,MxSmoker,MxStartDateDayOfMonth,MxStartDateDayOfWeek,MxStartDateMonth,MxWillReAdmit
0,12312,0.35,0,12312,Encounter for problem,1,0,0,0,0,...,5,2,13,0,5,0,24,3,12,0
1,13180,0.35,0,13180,Inpatient Visit,0,0,0,0,0,...,0,0,0,1,1,0,30,7,11,1
2,13181,0.57,1,13181,Inpatient Follow-Up,0,0,0,0,0,...,1,1,1,1,1,0,8,1,12,0
